# Random Forest - Detection of Swallowing Disorders

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas import ExcelWriter
import os
import numpy as np
import random
plt.style.use("seaborn")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

### Loading the dataset

In [2]:
healthy = pd.read_excel("D:/DATA SCIENCE/INTERNSHIP PROJECT/Modelling/HealthySamples.xlsx")
healthy = healthy[healthy["Sample status"] == "Perfect"]
healthy["Swallow_type"] = "healthy"
unhealthy = pd.read_excel("D:/DATA SCIENCE/INTERNSHIP PROJECT/Modelling/UnHealthySamples.xlsx")
unhealthy = unhealthy[unhealthy["Sample status"] == "Perfect"]
unhealthy["Swallow_type"] = "unhealthy"
data = healthy.append(unhealthy)[["Patient", "Sensor1_max", "Sensor2_max", "Sensor3_min", "Latency", "UES_start", "UES_end", "UES_Duration", "Swallow_type"]]
data["Label"] = np.where(data["Swallow_type"] == "healthy", 0, 1)
data

,Patient,Sensor1_max,Sensor2_max,Sensor3_min,Latency,UES_start,UES_end,UES_Duration,Swallow_type,Label
0,Control 1.txt,192.279698,113.474479,-20.601968,-0.028,1552.752,1553.392,0.640,healthy,0
1,Control 1.txt,167.821012,98.336004,-11.776150,-0.028,1782.184,1782.896,0.712,healthy,0
2,Control 1.txt,171.739528,249.844358,-24.181735,-0.036,611.556,612.284,0.728,healthy,0
3,Control 1.txt,200.377661,107.578393,-16.358434,-0.092,2333.764,2334.412,0.648,healthy,0
4,Control 1.txt,193.675898,105.770199,-25.335317,-0.172,1697.164,1697.784,0.620,healthy,0
...,...,...,...,...,...,...,...,...,...,...
194,HD11A.txt,114.937700,101.913100,-27.384550,0.211,117.157,117.828,0.671,unhealthy,1
195,HD11A.txt,195.568900,97.228870,-14.957330,0.200,163.116,164.111,0.995,unhealthy,1
196,HD11A.txt,200.185000,116.615400,-9.014693,-0.042,124.009,124.538,0.529,unhealthy,1
197,HD11B.txt,209.113400,79.385630,-12.359970,0.192,61.255,61.856,0.601,unhealthy,1


### Correlation matrix of Independent variables

In [3]:
corr = data[["Sensor1_max", "Sensor2_max", "Sensor3_min", "Latency", "UES_Duration"]].corr()
corr.style.background_gradient(cmap='coolwarm')

,Sensor1_max,Sensor2_max,Sensor3_min,Latency,UES_Duration
Sensor1_max,1,-0.0346022,-0.038871,0.149575,0.0296155
Sensor2_max,-0.0346022,1,-0.124292,0.0283056,0.056447
Sensor3_min,-0.038871,-0.124292,1,-0.0584012,0.063469
Latency,0.149575,0.0283056,-0.0584012,1,0.316552
UES_Duration,0.0296155,0.056447,0.063469,0.316552,1


### Training and Test data split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data[["Sensor1_max", "Sensor2_max", "Sensor3_min", "Latency", "UES_Duration"]], data[["Label"]], test_size=0.3, random_state=0)

### Composition of Training and test data

In [5]:
y_train.apply(pd.Series.value_counts)

,Label
0,86
1,84


In [6]:
y_test.apply(pd.Series.value_counts)

,Label
0,38
1,36


## Fitting a Random forest Model

In [7]:
model = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Comparing Predicted and Actual values

In [8]:
y_pred = model.predict(X_test)

In [9]:
pred = pd.DataFrame(y_pred)
pred["Predicted"] = np.where(pred[0] == 0, "Healthy", "Unhealthy")
pred["Actual"] = np.where(y_test["Label"] == 0, "Healthy", "Unhealthy")
pred[["Predicted", "Actual"]]

,Predicted,Actual
0,Healthy,Healthy
1,Unhealthy,Healthy
2,Healthy,Healthy
3,Healthy,Healthy
4,Unhealthy,Unhealthy
...,...,...
69,Healthy,Healthy
70,Healthy,Healthy
71,Healthy,Healthy
72,Healthy,Healthy


### Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
plt.figure(figsize = (7,7))
sns.set(font_scale=1.5)

[[26 12]
 [11 25]]


<Figure size 504x504 with 0 Axes>

### Model Evaluation metrics

In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.68      0.69        38
           1       0.68      0.69      0.68        36

    accuracy                           0.69        74
   macro avg       0.69      0.69      0.69        74
weighted avg       0.69      0.69      0.69        74



# Hyperparameter tuning in Random Forest

## Evaluating best performing parameters

In [12]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
param_grid = {'bootstrap': [True, False],
 'max_depth': [2, 3, 4, 5,10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [13]:
random_search = RandomizedSearchCV(model, param_distributions=param_grid, verbose = 3, cv=6, scoring="roc_auc")

In [14]:
random_search.fit(X_train, y_train)

Fitting 6 folds for each of 10 candidates, totalling 60 fits
[CV] n_estimators=1800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=True, score=0.814, total=   3.0s
[CV] n_estimators=1800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=True, score=0.719, total=   2.9s
[CV] n_estimators=1800, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=80, bootstrap=True 


...

[CV]  n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=70, bootstrap=True, score=0.658, total=   2.2s
[CV] n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=70, bootstrap=True 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=1400, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=70, bootstrap=True, score=0.602, total=   2.4s


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.8min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=6, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='sqrt',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=100,
                                                    n_jobs=None,
 

In [15]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1800,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Fitting a Random Forest with best performing parameters

In [16]:
model = random_search.best_estimator_
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1800,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Comparing Predicted and Actual values

In [17]:
y_pred = model.predict(X_test)

In [18]:
pred = pd.DataFrame(y_pred)
pred["Predicted"] = np.where(pred[0] == 0, "Healthy", "Unhealthy")
pred["Actual"] = np.where(y_test["Label"] == 0, "Healthy", "Unhealthy")
pred[["Predicted", "Actual"]]

,Predicted,Actual
0,Unhealthy,Healthy
1,Unhealthy,Healthy
2,Healthy,Healthy
3,Unhealthy,Healthy
4,Unhealthy,Unhealthy
...,...,...
69,Unhealthy,Healthy
70,Healthy,Healthy
71,Unhealthy,Healthy
72,Unhealthy,Healthy


## Recalculating Model Evaluation metrics for Model fitted with best performing parameters

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.53      0.58        38
           1       0.58      0.69      0.63        36

    accuracy                           0.61        74
   macro avg       0.61      0.61      0.61        74
weighted avg       0.61      0.61      0.61        74



In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
plt.figure(figsize = (7,7))
sns.set(font_scale=1.5)

[[20 18]
 [11 25]]


<Figure size 504x504 with 0 Axes>

## Changing threshold to achieve 100% detection of Unhealthy patients

In [68]:
THRESHOLD = 0.39
y_pred = np.where(model.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

## Reevaluating Model Evaluation metrics with new threshold

In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.21      0.35        38
           1       0.55      1.00      0.71        36

    accuracy                           0.59        74
   macro avg       0.77      0.61      0.53        74
weighted avg       0.78      0.59      0.52        74



In [72]:
from sklearn.metrics import confusion_matrix
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
plt.figure(figsize = (7,7))
sns.set(font_scale=1.5)

[[ 8 30]
 [ 0 36]]


<Figure size 504x504 with 0 Axes>